In [1]:
# Reading Dataset
import numpy as np
import pandas as pd

# Visualization
import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

from datetime import datetime

# Reading Dataset

In [2]:
# Start spark session
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.driver.memory", "10g").getOrCreate()

24/08/15 16:35:53 WARN Utils: Your hostname, ubuntu20 resolves to a loopback address: 127.0.1.1; using 192.168.0.234 instead (on interface wlp0s20f3)
24/08/15 16:35:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/15 16:35:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Load anime dataset
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType

schema = StructType([
    StructField("anime_id", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("English name", StringType(), True),
    StructField("Other name", StringType(), True),
    StructField("Score", FloatType(), True),
    StructField("Genres", StringType(), True),
    StructField("Synopsis", StringType(), True),
    StructField("Type", StringType(), True),
    StructField("Episodes", FloatType(), True),
    StructField("Aired", StringType(), True),
    StructField("Premiered", StringType(), True),
    StructField("Status", StringType(), True),
    StructField("Producers", StringType(), True),
    StructField("Licensors", StringType(), True),
    StructField("Studios", StringType(), True),
    StructField("Source", StringType(), True),
    StructField("Duration", StringType(), True),
    StructField("Rating", StringType(), True),
    StructField("Rank", FloatType(), True),
    StructField("Popularity", IntegerType(), True),
    StructField("Favorites", IntegerType(), True),
    StructField("Scored By", FloatType(), True),
    StructField("Members", FloatType(), True),
    StructField("Image URL", StringType(), True),
])

df_anime = spark.read.csv('../dataset/myanimelist-dataset/anime-dataset-2023.csv', header=True, schema=schema, multiLine=True, quote='\"', escape='\"')

In [4]:
df_anime \
            .filter((df_anime.Genres.contains('UNKNOWN')) | (df_anime.Studios.contains('UNKNOWN'))) \
            .count()

11682

In [4]:
df_anime.printSchema()

root
 |-- anime_id: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- English name: string (nullable = true)
 |-- Other name: string (nullable = true)
 |-- Score: float (nullable = true)
 |-- Genres: string (nullable = true)
 |-- Synopsis: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Episodes: float (nullable = true)
 |-- Aired: string (nullable = true)
 |-- Premiered: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Producers: string (nullable = true)
 |-- Licensors: string (nullable = true)
 |-- Studios: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Duration: string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Rank: float (nullable = true)
 |-- Popularity: integer (nullable = true)
 |-- Favorites: integer (nullable = true)
 |-- Scored By: float (nullable = true)
 |-- Members: float (nullable = true)
 |-- Image URL: string (nullable = true)



In [5]:
print("Shape of the Anime dataset:", df_anime.count(), len(df_anime.columns))
df_anime.show()

Shape of the Anime dataset: 24905 24
+--------+--------------------+--------------------+------------------------------+-----+--------------------+--------------------+-----+--------+--------------------+-----------+----------------+--------------------+--------------------+----------------+-----------+-------------+--------------------+------+----------+---------+---------+---------+--------------------+
|anime_id|                Name|        English name|                    Other name|Score|              Genres|            Synopsis| Type|Episodes|               Aired|  Premiered|          Status|           Producers|           Licensors|         Studios|     Source|     Duration|              Rating|  Rank|Popularity|Favorites|Scored By|  Members|           Image URL|
+--------+--------------------+--------------------+------------------------------+-----+--------------------+--------------------+-----+--------+--------------------+-----------+----------------+--------------------+--

In [4]:
# Importing user details dataset
schema = StructType([
    StructField("Mal ID", IntegerType(), True),
    StructField("Username", StringType(), True),
    StructField("Gender", StringType(), True),
    StructField("Birthday", StringType(), True),
    StructField("Location", StringType(), True),
    StructField("Joined", StringType(), True),
    StructField("Days Watched", FloatType(), True),
    StructField("Mean Score", FloatType(), True),
    StructField("Watching", FloatType(), True),
    StructField("Completed", FloatType(), True),
    StructField("On Hold", FloatType(), True),
    StructField("Dropped", FloatType(), True),
    StructField("Plan to Watch", FloatType(), True),
    StructField("Total Entries", FloatType(), True),
    StructField("Rewatched", FloatType(), True),
    StructField("Episodes Watched", FloatType(), True)
])

df_user = spark.read.csv("../dataset/myanimelist-dataset/users-details-2023.csv", header=True, schema=schema)

In [7]:
print("Shape of the User dataset:", df_user.count(), len(df_user.columns))
df_user.show()

Shape of the User dataset: 731290 16
+------+---------------+------+--------------------+--------------------+--------------------+------------+----------+--------+---------+-------+-------+-------------+-------------+---------+----------------+
|Mal ID|       Username|Gender|            Birthday|            Location|              Joined|Days Watched|Mean Score|Watching|Completed|On Hold|Dropped|Plan to Watch|Total Entries|Rewatched|Episodes Watched|
+------+---------------+------+--------------------+--------------------+--------------------+------------+----------+--------+---------+-------+-------+-------------+-------------+---------+----------------+
|     1|          Xinil|  Male|1985-03-04T00:00:...|          California|2004-11-05T00:00:...|       142.3|      7.37|     1.0|    233.0|    8.0|   93.0|         64.0|        399.0|     60.0|          8458.0|
|     3|        Aokaado|  Male|                NULL|        Oslo, Norway|2004-11-11T00:00:...|        68.6|      7.34|    23.0|

In [5]:
# Importing user score dataset
schema = StructType([
    StructField("user_id", IntegerType(), True),
    StructField("Username", StringType(), True),
    StructField("anime_id", IntegerType(), True),
    StructField("Anime Title", StringType(), True),
    StructField("rating", IntegerType(), True)
])

df_score = spark.read.csv("../dataset/myanimelist-dataset/users-score-2023.csv", header=True, schema=schema)

In [9]:
print("Shape of the Score dataset:", df_score.count(), len(df_score.columns))
df_score.show()

Shape of the Score dataset: 24325191 5
+-------+--------+--------+--------------------+------+
|user_id|Username|anime_id|         Anime Title|rating|
+-------+--------+--------+--------------------+------+
|      1|   Xinil|      21|           One Piece|     9|
|      1|   Xinil|      48|         .hack//Sign|     7|
|      1|   Xinil|     320|              A Kite|     5|
|      1|   Xinil|      49|    Aa! Megami-sama!|     8|
|      1|   Xinil|     304|Aa! Megami-sama! ...|     8|
|      1|   Xinil|     306|Abenobashi Mahou☆...|     8|
|      1|   Xinil|      53|       Ai Yori Aoshi|     7|
|      1|   Xinil|      47|               Akira|     5|
|      1|   Xinil|     591|      Amaenaide yo!!|     6|
|      1|   Xinil|      54|   Appleseed (Movie)|     7|
|      1|   Xinil|      55|         Arc the Lad|     5|
|      1|   Xinil|      56|             Avenger|     6|
|      1|   Xinil|      57|                Beck|     9|
|      1|   Xinil|     368|         Bible Black|     5|
|      1|

# Data Exploration

## Anime Dataset

In [10]:
df_anime.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24905 entries, 0 to 24904
Data columns (total 24 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   anime_id      24905 non-null  int32  
 1   Name          24905 non-null  object 
 2   English name  24905 non-null  object 
 3   Other name    24905 non-null  object 
 4   Score         15692 non-null  float32
 5   Genres        24905 non-null  object 
 6   Synopsis      24905 non-null  object 
 7   Type          24905 non-null  object 
 8   Episodes      24294 non-null  float32
 9   Aired         24905 non-null  object 
 10  Premiered     24905 non-null  object 
 11  Status        24905 non-null  object 
 12  Producers     24905 non-null  object 
 13  Licensors     24905 non-null  object 
 14  Studios       24905 non-null  object 
 15  Source        24905 non-null  object 
 16  Duration      24905 non-null  object 
 17  Rating        24905 non-null  object 
 18  Rank          20293 non-nu

In [4]:
from pyspark.sql import functions as F

df_anime.select('Score').dropna().count()

15692

In [12]:
df_anime.describe('Score').show()

+-------+------------------+
|summary|             Score|
+-------+------------------+
|  count|             15692|
|   mean|6.3808896258462005|
| stddev|0.9286960614060261|
|    min|              1.85|
|    max|               9.1|
+-------+------------------+



In [6]:
average_score = df_anime.agg(F.round(F.mean(F.col('Score')), 2).alias('Avg_Score')).first().Avg_Score
average_score

6.38

In [7]:
df_anime = df_anime.fillna({'Score': average_score})

In [10]:
df_anime.describe('Score').show()

+-------+------------------+
|summary|             Score|
+-------+------------------+
|  count|             24905|
|   mean|6.3805605727011745|
| stddev|0.7371645377844217|
|    min|              1.85|
|    max|               9.1|
+-------+------------------+



In [16]:
dict_null = { col : df_anime.filter(df_anime[col].isNull()).count() for col in df_anime.columns }
dict_null

{'anime_id': 0,
 'Name': 0,
 'English name': 0,
 'Other name': 0,
 'Score': 0,
 'Genres': 0,
 'Synopsis': 0,
 'Type': 0,
 'Episodes': 611,
 'Aired': 0,
 'Premiered': 0,
 'Status': 0,
 'Producers': 0,
 'Licensors': 0,
 'Studios': 0,
 'Source': 0,
 'Duration': 0,
 'Rating': 0,
 'Rank': 4612,
 'Popularity': 0,
 'Favorites': 0,
 'Scored By': 9213,
 'Members': 0,
 'Image URL': 0}

In [40]:
df_anime.toPandas().to_csv("../dataset/myanimelist-dataset/processed-dataset/anime-dataset-2023.csv", index=False)

## User Details Dataset

In [17]:
df_user.toPandas().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731290 entries, 0 to 731289
Data columns (total 16 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Mal ID            731290 non-null  int32  
 1   Username          731290 non-null  object 
 2   Gender            224383 non-null  object 
 3   Birthday          168068 non-null  object 
 4   Location          152806 non-null  object 
 5   Joined            731290 non-null  object 
 6   Days Watched      731272 non-null  float32
 7   Mean Score        731281 non-null  float32
 8   Watching          731282 non-null  float32
 9   Completed         731282 non-null  float32
 10  On Hold           731282 non-null  float32
 11  Dropped           731282 non-null  float32
 12  Plan to Watch     731282 non-null  float32
 13  Total Entries     731282 non-null  float32
 14  Rewatched         731282 non-null  float32
 15  Episodes Watched  731282 non-null  float32
dtypes: float32(10), int3

In [11]:
dict_null = { col : df_user.filter(df_user[col].isNull()).count() for col in df_user.columns }
dict_null

{'Mal ID': 0,
 'Username': 0,
 'Gender': 506907,
 'Birthday': 563222,
 'Location': 578484,
 'Joined': 0,
 'Days Watched': 18,
 'Mean Score': 9,
 'Watching': 8,
 'Completed': 8,
 'On Hold': 8,
 'Dropped': 8,
 'Plan to Watch': 8,
 'Total Entries': 8,
 'Rewatched': 8,
 'Episodes Watched': 8}

In [19]:
user_entry_null_ids = df_user.filter(df_user['Total Entries'].isNull()).select('Mal ID').collect()
user_entry_null_ids = [int(user_id[0]) for user_id in user_entry_null_ids]
user_entry_null_ids

[8682, 79090, 125668, 160113, 231825, 232646, 277237, 561555]

In [20]:
user_entry_null_rating_count = df_score.filter(df_score['user_id'].isin(user_entry_null_ids)).count()
all_ratings_count = df_score.count()
print(
    "Number of ratings of user with null entry: ",
    user_entry_null_rating_count, " ==> ", 
    user_entry_null_rating_count / all_ratings_count * 100
)

Number of ratings of user with null entry:  330  ==>  0.001356618330355556


In [21]:
df_user = df_user.filter(df_user['Mal ID'].isin(user_entry_null_ids) == False)
df_user.count()

731282

In [22]:
dict_null = { col : df_user.filter(df_user[col].isNull()).count() for col in df_user.columns }
dict_null

{'Mal ID': 0,
 'Username': 0,
 'Gender': 506907,
 'Birthday': 563220,
 'Location': 578482,
 'Joined': 0,
 'Days Watched': 10,
 'Mean Score': 1,
 'Watching': 0,
 'Completed': 0,
 'On Hold': 0,
 'Dropped': 0,
 'Plan to Watch': 0,
 'Total Entries': 0,
 'Rewatched': 0,
 'Episodes Watched': 0}

In [23]:
df_user.toPandas().to_csv("../dataset/myanimelist-dataset/processed-dataset/users-details-2023.csv", index=False)

## User Score Dataset

In [24]:
# Count number of null values in each column
dict_null = { col : df_score.filter(df_score[col].isNull()).count() for col in df_score.columns }
dict_null

{'user_id': 0, 'Username': 0, 'anime_id': 0, 'Anime Title': 0, 'rating': 3127}

In [25]:
df_score.filter(df_score['rating'].isNull()).show()

+-------+---------------+--------+--------------------+------+
|user_id|       Username|anime_id|         Anime Title|rating|
+-------+---------------+--------+--------------------+------+
|    357|         zhambi|   31630|"Gyakuten Saiban:...|  NULL|
|    433|    SleepySmore|   31630|"Gyakuten Saiban:...|  NULL|
|    433|    SleepySmore|   37490|"Gyakuten Saiban:...|  NULL|
|    651|          Artic|   31630|"Gyakuten Saiban:...|  NULL|
|    791|      Kuisanagi|   31630|"Gyakuten Saiban:...|  NULL|
|    835|    Night_Demon|   31630|"Gyakuten Saiban:...|  NULL|
|    840| Dragon_Empress|   31630|"Gyakuten Saiban:...|  NULL|
|    840| Dragon_Empress|   37490|"Gyakuten Saiban:...|  NULL|
|    859|        Sora_Li|   31630|"Gyakuten Saiban:...|  NULL|
|    916|thesundaywriter|   31630|"Gyakuten Saiban:...|  NULL|
|   1031|   lugiastrikes|   31630|"Gyakuten Saiban:...|  NULL|
|   1125|      Hechizero|   31630|"Gyakuten Saiban:...|  NULL|
|   1125|      Hechizero|   37490|"Gyakuten Saiban:...|

In [26]:
df_score = df_score.na.drop(subset=['rating'])

In [27]:
df_score = df_score.filter(df_score['user_id'].isin(user_entry_null_ids) == False)

In [28]:
df_score.count()

24321734

In [29]:
# Count number of null values in each column
dict_null = { col : df_score.filter(df_score[col].isNull()).count() for col in df_score.columns }
dict_null

{'user_id': 0, 'Username': 0, 'anime_id': 0, 'Anime Title': 0, 'rating': 0}

In [31]:
df_score.coalesce(1).write.option("header", True).csv("../dataset/myanimelist-dataset/processed-dataset/users-score-2023.csv")

# Data Vizualization

## Anime dataset

In [8]:
import matplotlib.pyplot as plt

In [12]:
# Count anime by type
test_df = df_anime.groupBy('Type').count()
test_df.show()

+-------+-----+
|   Type|count|
+-------+-----+
|     TV| 7597|
|Special| 2558|
|UNKNOWN|   74|
|    OVA| 4076|
|  Music| 2686|
|  Movie| 4381|
|    ONA| 3533|
+-------+-----+



In [13]:
test_df.select(F.collect_list('count')).first()[0]

[7597, 2558, 74, 4076, 2686, 4381, 3533]

In [5]:
# Count the number of anime titles by type
type_counts = df_anime.groupBy('Type').count().orderBy('count', ascending=False)
types = type_counts.select(F.collect_list('Type')).first()[0]
count_by_types = type_counts.select(F.collect_list('count')).first()[0]

# Create a bar chart
fig = px.bar(x=types, y=count_by_types, color=types, labels={'x':'Anime Type', 'y':'Count'}, 
             title='Count of Anime by Type')

fig.show()

In [7]:
from pyspark.sql.types import ArrayType
genres_counts = df_anime \
    .select('Genres') \
    .filter(~F.col('Genres').contains('UNKNOWN')) \
    .withColumn('Genres', F.split(F.col('Genres'), ', ').cast(ArrayType(StringType()))) \
    .select(F.explode(F.col('Genres')).alias('Genres')) \
    .groupBy('Genres').count() \
    .orderBy('count', ascending=False)

genres = genres_counts.select(F.collect_list('Genres')).first()[0]
count_by_genres = genres_counts.select(F.collect_list('count')).first()[0]

# Create a bar chart
fig = px.bar(x=genres, y=count_by_genres, color=genres, labels={'x':'Anime Genres', 'y':'Count'}, 
             title='Count of Anime by Genres')

fig.show()

In [10]:
corr_df = df_anime.select('Score', 'Popularity', 'Rank').toPandas()

In [11]:
import plotly.figure_factory as ff

corr_matrix = corr_df.corr()
fig = ff.create_annotated_heatmap(z=corr_matrix.values,
                                  x=list(corr_matrix.columns),
                                  y=list(corr_matrix.index),
                                  colorscale='Magma')
fig.update_layout(title='Correlation among Score, Popularity and Rank')
fig.show()

In [12]:
fig.write_image("corr.png")